# Goals
1. Download ONNX model + config from HL
2. Get relevant parameters (e.g. input size, class list) out of config or model file
3. Download image from HL
4. ~Instantiate ONNX model~
4. ~Perform inference on image~
5. ~Display results (save to file)~

# House Keeping

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Load ONNX model and perform inference

In [ ]:
onnx_model_file = "insulator_tie_model_epoch_15.onnx"
config_file = "fixed-model-config.py"
img_file = "/mnt/scratch/experiments/images/5568090.jpg"
img_file_2 = '/mnt/scratch/experiments/images/5567791.jpg'
output_inferences_file = 'show_inferences.jpg'

In [ ]:
import numpy as np
import onnx
from mmcv import Config
from mmdet.core.export import preprocess_example_input
from mmdet.core.export.model_wrappers import ONNXRuntimeDetector

In [ ]:
onnx_config = Config.fromfile(config_file)

# mystery: where do the 800 and 1216 come from?
# they're not the image scale in the config - that's (2400, 3200)
input_config = {'input_shape': (1,3,
                                800, # onnx_config.scales[0]['image_size'][0], 
                                1216 ), #onnx_config.scales[0]['image_size'][1]),
                'input_path': img_file,
                'normalize_cfg':onnx_config.img_norm_cfg,}

one_img, one_meta = preprocess_example_input(input_config)
img_list, img_meta_list = [one_img], [[one_meta]]
img_list = [_.cuda().contiguous() for _ in img_list]

In [ ]:
# Mystery: the first time you run this cell it appears to hang. You have to
# interrupt the kernel and rerun it, at which point it will finish fast
onnx_model = ONNXRuntimeDetector(onnx_model_file, 
                                 class_names=np.array(['Insulator', 'Tie Wire']), 
                                 device_id=0)

onnx_results = onnx_model(img_list, img_metas=img_meta_list, return_loss=False)[0]

In [ ]:
show_img = one_meta['show_img']
score_thr=0.5
onnx_model.show_result(
            show_img,
            onnx_results,
            score_thr=score_thr,
            show=True,
            win_name='ONNXRuntime',
            out_file=output_inferences_file)

# Create a HLClient object from credentials

This client will be used when we need to communicate with Highlighter via GraphQL.

In [ ]:
HL_WEB_GRAPHQL_API_TOKEN="..."
HL_WEB_GRAPHQL_ENDPOINT="https://<ACCOUNT_NAME>.highlighter.ai/graphql"

In [ ]:
from highlighter_client.gql_client import HLClient

# Needed when using HighlighterClient in a notebook environment
HLClient._async = True

# Small helper function for displaying the DataFrames in the highlighter clinet
# dataset object
def display_ds(ds, count=10):
    display(ds.annotations_df.head(count))
    display(ds.images_df.head(count))

In [ ]:
client = HLClient.from_credential(api_token=HL_WEB_GRAPHQL_API_TOKEN, endpoint_url=HL_WEB_GRAPHQL_ENDPOINT)

# Read Dataset from Highlighter

`HighlighterClient` represents datasets as two Pandas DataFrames `annotations_df` and `images_df`. We can populate a `HighlighterClient.Dataset` in several ways using `Readers`. You can list the availaible `Readers` and load one from its name. In this case we'll be loading the `HighlighterSubmissionsReader` so we can pull submissions down from Highlighter.

In [ ]:
from highlighter_client.datasets import get_reader, READERS

print(f"READERS: {list(READERS.keys())}")

reader = get_reader("highlighter_submissions")()

In [ ]:
# View the doc string and function signature
# Note it expects a submissions generator
# We will create one in a moment.
?reader

Once we have a `Reader` we can initialize a `highlighter_client.Dataset` object 
and with that `Reader`

In [ ]:
from highlighter_client.datasets.dataset import Dataset
ds = Dataset(reader=reader)

Now we have a `highlighter_client.Dataset` with a `HighlighterSubmissionsReader` we can populate our `DataFrames`.

To understand this we need to know two things.

1. `highlighter_client` uses Pandas `BaseModel` to tell GraphQL what values to return from a query. Some common `BaseModel`s are defined in `highlighter_client.base_models` but if you want more fine grained control you can define your own.

2. Some GraphQL queries may return many results. These types of queries are called `Connections` are are named accordingly in the code. There is a `paginate` function that takes a `Connection` query and returns a Python Generator.

For more information on the BaseModels see `highlighter_client/base_models.py`

In [ ]:
from highlighter_client.base_models import DatasetSubmissionTypeConnection
from highlighter_client.paginate import paginate

dataset_id = ?

submissions_gen = paginate(
client.datasetSubmissionConnection,
DatasetSubmissionTypeConnection,
datasetId=dataset_id,
)

ds.read(submissions_gen=submissions_gen)
display_ds(ds)